# 🤖 NFL Player Movement - Model Comparison

**Comprehensive Model Selection and Ensemble Methods**

This notebook compares multiple machine learning models for player movement prediction and explores ensemble techniques.

---

## 📋 Table of Contents

1. [Setup & Configuration](#1-setup)
2. [Data Loading & Preparation](#2-data)
3. [Train/Validation Split](#3-split)
4. [Model Training](#4-training)
5. [Cross-Validation](#5-cv)
6. [Hyperparameter Tuning](#6-tuning)
7. [Model Comparison](#7-comparison)
8. [Ensemble Methods](#8-ensemble)
9. [Best Model Selection](#9-selection)

---

## 1. Setup & Configuration 🔧

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
import pickle
import time
from datetime import datetime
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Try to import XGBoost and LightGBM
try:
    from xgboost import XGBRegressor
    HAS_XGBOOST = True
except ImportError:
    HAS_XGBOOST = False
    print("⚠️  XGBoost not installed. Install with: pip install xgboost")

try:
    import lightgbm as lgb
    HAS_LIGHTGBM = True
except ImportError:
    HAS_LIGHTGBM = False
    print("⚠️  LightGBM not installed. Install with: pip install lightgbm")

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries imported successfully")
print(f"   XGBoost available: {HAS_XGBOOST}")
print(f"   LightGBM available: {HAS_LIGHTGBM}")

In [ ]:
# Configuration
class Config:
    """Model comparison configuration"""
    
    # Paths
    DATA_DIR = Path('../data/raw/train')
    OUTPUT_DIR = Path('../outputs/model_comparison')
    
    # Data settings
    USE_SAMPLE = True
    SAMPLE_SIZE = 50000
    MAX_FILES = 2
    
    # Split settings
    VAL_SIZE = 0.2
    RANDOM_STATE = 42
    
    # Cross-validation
    N_FOLDS = 5
    
    # Models to compare
    MODELS = ['ridge', 'lasso', 'elastic_net', 'random_forest', 'gradient_boosting']
    if HAS_XGBOOST:
        MODELS.append('xgboost')
    if HAS_LIGHTGBM:
        MODELS.append('lightgbm')

config = Config()
config.OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")
print(f"   Models to compare: {len(config.MODELS)}")
print(f"   Models: {config.MODELS}")

## 2. Data Loading & Preparation 📂

In [ ]:
def load_and_prepare_data(data_dir, max_files=None, sample_size=None):
    """
    Load data and create features
    
    Returns:
        df: Dataframe with features and targets
    """
    print("📂 Loading data...\n")
    
    # Get file lists
    input_files = sorted(data_dir.glob('input_*.csv'))[:max_files] if max_files else sorted(data_dir.glob('input_*.csv'))
    output_files = sorted(data_dir.glob('output_*.csv'))[:max_files] if max_files else sorted(data_dir.glob('output_*.csv'))
    
    # Load
    input_df = pd.concat([pd.read_csv(f) for f in input_files], ignore_index=True)
    output_df = pd.concat([pd.read_csv(f) for f in output_files], ignore_index=True)
    
    print(f"   Input: {input_df.shape}")
    print(f"   Output: {output_df.shape}")
    
    # Sample
    if sample_size and len(input_df) > sample_size:
        input_df = input_df.sample(n=sample_size, random_state=42)
        sampled_keys = input_df[['game_id', 'play_id', 'nfl_id', 'frame_id']]
        output_df = output_df.merge(sampled_keys, on=['game_id', 'play_id', 'nfl_id', 'frame_id'])
    
    # Merge
    df = input_df.merge(
        output_df[['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y']],
        on=['game_id', 'play_id', 'nfl_id', 'frame_id'],
        suffixes=('', '_target')
    )
    df = df.rename(columns={'x_target': 'target_x', 'y_target': 'target_y'})
    
    # Handle missing values
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df[col].isnull().any():
            df[col].fillna(df[col].median(), inplace=True)
    
    print(f"\n✅ Data loaded and merged: {df.shape}")
    return df


def create_features(df):
    """
    Create all features (physics, spatial, NFL)
    """
    print("\n⚙️  Creating features...\n")
    df_copy = df.copy()
    
    # Physics features
    if 's' in df_copy.columns and 'dir' in df_copy.columns:
        df_copy['velocity_x'] = df_copy['s'] * np.cos(np.radians(df_copy['dir']))
        df_copy['velocity_y'] = df_copy['s'] * np.sin(np.radians(df_copy['dir']))
        print("   ✓ Physics features")
    
    if 'a' in df_copy.columns and 'dir' in df_copy.columns:
        df_copy['acceleration_x'] = df_copy['a'] * np.cos(np.radians(df_copy['dir']))
        df_copy['acceleration_y'] = df_copy['a'] * np.sin(np.radians(df_copy['dir']))
    
    if 'player_weight' in df_copy.columns and 's' in df_copy.columns:
        df_copy['momentum'] = df_copy['player_weight'] * df_copy['s']
        df_copy['kinetic_energy'] = 0.5 * df_copy['player_weight'] * (df_copy['s'] ** 2)
    
    # Spatial features
    if all(col in df_copy.columns for col in ['x', 'y', 'ball_land_x', 'ball_land_y']):
        df_copy['dist_to_ball'] = np.sqrt((df_copy['x'] - df_copy['ball_land_x'])**2 + (df_copy['y'] - df_copy['ball_land_y'])**2)
        df_copy['dx_to_ball'] = df_copy['ball_land_x'] - df_copy['x']
        df_copy['dy_to_ball'] = df_copy['ball_land_y'] - df_copy['y']
        print("   ✓ Spatial features")
    
    if 'x' in df_copy.columns:
        df_copy['field_position_norm'] = df_copy['x'] / 120.0
        df_copy['dist_to_sideline'] = np.minimum(df_copy['y'], 53.3 - df_copy['y'])
    
    # NFL features
    if 'player_role' in df_copy.columns:
        df_copy['is_targeted_receiver'] = (df_copy['player_role'] == 'Targeted Receiver').astype(int)
        df_copy['is_passer'] = (df_copy['player_role'] == 'Passer').astype(int)
        print("   ✓ NFL domain features")
    
    print(f"\n✅ Features created: {len([c for c in df_copy.columns if c not in df.columns])} new")
    return df_copy

In [ ]:
# Load and prepare data
df = load_and_prepare_data(
    config.DATA_DIR,
    max_files=config.MAX_FILES,
    sample_size=config.SAMPLE_SIZE if config.USE_SAMPLE else None
)

df = create_features(df)

## 3. Train/Validation Split 🔀

In [ ]:
def prepare_model_data(df, val_size=0.2, random_state=42):
    """
    Prepare features and split into train/validation
    
    Returns:
        X_train, X_val, y_train_x, y_val_x, y_train_y, y_val_y, feature_names
    """
    print("\n🔀 Preparing model data...\n")
    
    # Identify feature columns
    exclude_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'target_x', 'target_y',
                    'player_name', 'player_position', 'player_role', 'player_side',
                    'play_direction', 'player_birth_date', 'player_to_predict']
    
    feature_cols = [col for col in df.columns if col not in exclude_cols and df[col].dtype in ['int64', 'float64']]
    
    # Prepare features and targets
    X = df[feature_cols].fillna(0)
    y_x = df['target_x'].fillna(df['target_x'].median())
    y_y = df['target_y'].fillna(df['target_y'].median())
    
    # Temporal split by game (if available)
    if 'game_id' in df.columns:
        unique_games = df['game_id'].unique()
        np.random.seed(random_state)
        np.random.shuffle(unique_games)
        
        split_idx = int(len(unique_games) * (1 - val_size))
        train_games = unique_games[:split_idx]
        val_games = unique_games[split_idx:]
        
        train_mask = df['game_id'].isin(train_games)
        val_mask = df['game_id'].isin(val_games)
        
        X_train, X_val = X[train_mask], X[val_mask]
        y_train_x, y_val_x = y_x[train_mask], y_x[val_mask]
        y_train_y, y_val_y = y_y[train_mask], y_y[val_mask]
        
        print("   Split type: Temporal (by game)")
    else:
        # Random split
        from sklearn.model_selection import train_test_split
        X_train, X_val, y_train_x, y_val_x = train_test_split(X, y_x, test_size=val_size, random_state=random_state)
        _, _, y_train_y, y_val_y = train_test_split(X, y_y, test_size=val_size, random_state=random_state)
        print("   Split type: Random")
    
    print(f"   Train: {X_train.shape[0]:,} samples")
    print(f"   Val: {X_val.shape[0]:,} samples")
    print(f"   Features: {X_train.shape[1]}")
    
    return X_train, X_val, y_train_x, y_val_x, y_train_y, y_val_y, feature_cols


# Split data
X_train, X_val, y_train_x, y_val_x, y_train_y, y_val_y, feature_names = prepare_model_data(
    df, val_size=config.VAL_SIZE, random_state=config.RANDOM_STATE
)

print("\n✅ Data prepared for modeling")

## 4. Model Training 🤖

Train multiple models and compare performance:
- **Linear Models**: Ridge, Lasso, ElasticNet
- **Tree Models**: Random Forest, Gradient Boosting
- **Boosting Models**: XGBoost, LightGBM (if available)

In [ ]:
def get_model(model_name):
    """
    Get model instance by name
    """
    if model_name == 'ridge':
        return Ridge(alpha=1.0, random_state=42)
    elif model_name == 'lasso':
        return Lasso(alpha=1.0, random_state=42, max_iter=2000)
    elif model_name == 'elastic_net':
        return ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42, max_iter=2000)
    elif model_name == 'random_forest':
        return RandomForestRegressor(n_estimators=100, max_depth=15, min_samples_split=10, random_state=42, n_jobs=-1)
    elif model_name == 'gradient_boosting':
        return GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
    elif model_name == 'xgboost' and HAS_XGBOOST:
        return XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1)
    elif model_name == 'lightgbm' and HAS_LIGHTGBM:
        return lgb.LGBMRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, random_state=42, n_jobs=-1, verbose=-1)
    else:
        raise ValueError(f"Unknown model: {model_name}")


def train_and_evaluate(model, X_train, y_train, X_val, y_val, model_name):
    """
    Train model and evaluate performance
    
    Returns:
        model, metrics, predictions, training_time
    """
    # Train
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Predict
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    
    # Metrics
    metrics = {
        'model': model_name,
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'val_rmse': np.sqrt(mean_squared_error(y_val, y_pred_val)),
        'train_mae': mean_absolute_error(y_train, y_pred_train),
        'val_mae': mean_absolute_error(y_val, y_pred_val),
        'train_r2': r2_score(y_train, y_pred_train),
        'val_r2': r2_score(y_val, y_pred_val),
        'training_time': training_time
    }
    
    return model, metrics, y_pred_val, training_time

In [ ]:
# Train all models for X coordinate
print("="*70)
print("TRAINING MODELS - X COORDINATE")
print("="*70 + "\n")

models_x = {}
metrics_x = []
predictions_x = {}

for model_name in config.MODELS:
    print(f"🤖 Training {model_name.upper()}...")
    model = get_model(model_name)
    model, metrics, pred, train_time = train_and_evaluate(model, X_train, y_train_x, X_val, y_val_x, model_name)
    
    models_x[model_name] = model
    metrics_x.append(metrics)
    predictions_x[model_name] = pred
    
    print(f"   ✓ Val RMSE: {metrics['val_rmse']:.4f} | Val R²: {metrics['val_r2']:.4f} | Time: {train_time:.2f}s\n")

print("✅ X coordinate models trained\n")

In [ ]:
# Train all models for Y coordinate
print("="*70)
print("TRAINING MODELS - Y COORDINATE")
print("="*70 + "\n")

models_y = {}
metrics_y = []
predictions_y = {}

for model_name in config.MODELS:
    print(f"🤖 Training {model_name.upper()}...")
    model = get_model(model_name)
    model, metrics, pred, train_time = train_and_evaluate(model, X_train, y_train_y, X_val, y_val_y, model_name)
    
    models_y[model_name] = model
    metrics_y.append(metrics)
    predictions_y[model_name] = pred
    
    print(f"   ✓ Val RMSE: {metrics['val_rmse']:.4f} | Val R²: {metrics['val_r2']:.4f} | Time: {train_time:.2f}s\n")

print("✅ Y coordinate models trained\n")

## 5. Cross-Validation 🔄

Perform K-Fold and Time-Series cross-validation for robust evaluation.

In [ ]:
def cross_validate_model(model_name, X, y, cv_type='kfold', n_splits=5):
    """
    Perform cross-validation
    
    Args:
        model_name: Name of model
        X, y: Features and target
        cv_type: 'kfold' or 'timeseries'
        n_splits: Number of folds
    
    Returns:
        cv_scores: List of RMSE scores for each fold
    """
    # Select CV strategy
    if cv_type == 'kfold':
        cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    else:
        cv = TimeSeriesSplit(n_splits=n_splits)
    
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X), 1):
        X_train_cv = X.iloc[train_idx]
        X_val_cv = X.iloc[val_idx]
        y_train_cv = y.iloc[train_idx]
        y_val_cv = y.iloc[val_idx]
        
        # Train and evaluate
        model = get_model(model_name)
        model.fit(X_train_cv, y_train_cv)
        y_pred = model.predict(X_val_cv)
        rmse = np.sqrt(mean_squared_error(y_val_cv, y_pred))
        cv_scores.append(rmse)
    
    return cv_scores

In [ ]:
# Perform K-Fold CV for top 3 models (faster)
print("🔄 Performing K-Fold Cross-Validation...\n")

# Select top 3 models based on validation RMSE
top_3_models = sorted(metrics_x, key=lambda x: x['val_rmse'])[:3]
top_3_names = [m['model'] for m in top_3_models]

cv_results = {}

for model_name in top_3_names:
    print(f"   {model_name.upper()}...")
    cv_scores = cross_validate_model(model_name, X_train, y_train_x, cv_type='kfold', n_splits=config.N_FOLDS)
    cv_results[model_name] = cv_scores
    print(f"      Mean RMSE: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}\n")

print("✅ Cross-validation complete")

In [ ]:
# Visualize CV results
fig, ax = plt.subplots(figsize=(12, 6))

cv_data = [cv_results[name] for name in top_3_names]
positions = range(1, len(top_3_names) + 1)

bp = ax.boxplot(cv_data, positions=positions, labels=[n.upper() for n in top_3_names],
                patch_artist=True, widths=0.6)

# Color boxes
colors = ['lightblue', 'lightgreen', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel('RMSE', fontsize=12)
ax.set_title(f'{config.N_FOLDS}-Fold Cross-Validation Results', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'cross_validation.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Cross-validation visualized")

## 6. Hyperparameter Tuning 🎛️

Perform grid search for hyperparameter optimization.

In [ ]:
# Example: Tune Random Forest
print("🎛️  Tuning Random Forest hyperparameters...\n")

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 15, 20],
    'min_samples_split': [10, 20]
}

# Use smaller sample for faster tuning
sample_size = min(10000, len(X_train))
sample_idx = np.random.choice(len(X_train), sample_size, replace=False)
X_tune = X_train.iloc[sample_idx]
y_tune = y_train_x.iloc[sample_idx]

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(rf_base, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_tune, y_tune)

print(f"\n✅ Best parameters: {grid_search.best_params_}")
print(f"   Best RMSE: {np.sqrt(-grid_search.best_score_):.4f}")

# Save tuning results
tuning_results = pd.DataFrame(grid_search.cv_results_)
tuning_results.to_csv(config.OUTPUT_DIR / 'hyperparameter_tuning.csv', index=False)

## 7. Model Comparison 📊

Compare all models across multiple metrics and visualize results.

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame(metrics_x)
comparison_df_y = pd.DataFrame(metrics_y)

# Merge X and Y metrics
comparison = pd.merge(
    comparison_df[['model', 'val_rmse', 'val_mae', 'val_r2', 'training_time']],
    comparison_df_y[['model', 'val_rmse', 'val_mae', 'val_r2']],
    on='model',
    suffixes=('_x', '_y')
)

# Add average RMSE
comparison['avg_rmse'] = (comparison['val_rmse_x'] + comparison['val_rmse_y']) / 2
comparison = comparison.sort_values('avg_rmse')

print("="*70)
print("MODEL COMPARISON")
print("="*70 + "\n")
display(comparison)

# Save comparison
comparison.to_csv(config.OUTPUT_DIR / 'model_comparison.csv', index=False)
print(f"\n✅ Comparison saved to: {config.OUTPUT_DIR / 'model_comparison.csv'}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

models_list = comparison['model'].tolist()
x_pos = np.arange(len(models_list))

# 1. RMSE comparison
axes[0, 0].bar(x_pos - 0.2, comparison['val_rmse_x'], width=0.4, label='X coordinate', alpha=0.8)
axes[0, 0].bar(x_pos + 0.2, comparison['val_rmse_y'], width=0.4, label='Y coordinate', alpha=0.8)
axes[0, 0].set_xticks(x_pos)
axes[0, 0].set_xticklabels([m.upper() for m in models_list], rotation=45, ha='right')
axes[0, 0].set_ylabel('RMSE', fontsize=12)
axes[0, 0].set_title('Model RMSE Comparison', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. MAE comparison
axes[0, 1].bar(x_pos - 0.2, comparison['val_mae_x'], width=0.4, label='X coordinate', alpha=0.8, color='orange')
axes[0, 1].bar(x_pos + 0.2, comparison['val_mae_y'], width=0.4, label='Y coordinate', alpha=0.8, color='red')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_xticklabels([m.upper() for m in models_list], rotation=45, ha='right')
axes[0, 1].set_ylabel('MAE', fontsize=12)
axes[0, 1].set_title('Model MAE Comparison', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. R² comparison
axes[1, 0].bar(x_pos - 0.2, comparison['val_r2_x'], width=0.4, label='X coordinate', alpha=0.8, color='green')
axes[1, 0].bar(x_pos + 0.2, comparison['val_r2_y'], width=0.4, label='Y coordinate', alpha=0.8, color='cyan')
axes[1, 0].set_xticks(x_pos)
axes[1, 0].set_xticklabels([m.upper() for m in models_list], rotation=45, ha='right')
axes[1, 0].set_ylabel('R² Score', fontsize=12)
axes[1, 0].set_title('Model R² Comparison', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Training time
axes[1, 1].bar(x_pos, comparison['training_time'], alpha=0.8, color='purple')
axes[1, 1].set_xticks(x_pos)
axes[1, 1].set_xticklabels([m.upper() for m in models_list], rotation=45, ha='right')
axes[1, 1].set_ylabel('Training Time (seconds)', fontsize=12)
axes[1, 1].set_title('Model Training Time Comparison', fontsize=14, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'model_comparison_charts.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Model comparison visualized")

In [ ]:
# Feature importance comparison (for tree-based models)
print("\n🌳 Comparing feature importance across tree-based models...\n")

tree_models = ['random_forest', 'gradient_boosting']
if HAS_XGBOOST:
    tree_models.append('xgboost')
if HAS_LIGHTGBM:
    tree_models.append('lightgbm')

# Get available tree models
available_tree_models = [m for m in tree_models if m in models_x]

if available_tree_models:
    fig, axes = plt.subplots(1, len(available_tree_models), figsize=(6*len(available_tree_models), 6))
    if len(available_tree_models) == 1:
        axes = [axes]
    
    for idx, model_name in enumerate(available_tree_models):
        model = models_x[model_name]
        
        # Get feature importances
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            top_n = 15
            indices = np.argsort(importances)[-top_n:]
            
            axes[idx].barh(range(top_n), importances[indices], alpha=0.8)
            axes[idx].set_yticks(range(top_n))
            axes[idx].set_yticklabels([feature_names[i] for i in indices], fontsize=9)
            axes[idx].set_xlabel('Importance', fontsize=11)
            axes[idx].set_title(f'{model_name.upper()}\nTop 15 Features', fontsize=12, fontweight='bold')
            axes[idx].grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(config.OUTPUT_DIR / 'feature_importance_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ Feature importance compared")
else:
    print("⚠️  No tree-based models available for feature importance comparison")

## 8. Ensemble Methods 🎭

Combine multiple models using ensemble techniques:
- Simple averaging
- Weighted averaging
- Stacking

In [ ]:
print("🎭 Creating ensemble predictions...\n")

# 1. Simple averaging ensemble
ensemble_simple_x = np.mean([predictions_x[m] for m in config.MODELS], axis=0)
ensemble_simple_y = np.mean([predictions_y[m] for m in config.MODELS], axis=0)

rmse_ensemble_simple_x = np.sqrt(mean_squared_error(y_val_x, ensemble_simple_x))
rmse_ensemble_simple_y = np.sqrt(mean_squared_error(y_val_y, ensemble_simple_y))

print(f"📊 Simple Averaging Ensemble:")
print(f"   X RMSE: {rmse_ensemble_simple_x:.4f}")
print(f"   Y RMSE: {rmse_ensemble_simple_y:.4f}")
print(f"   Average: {(rmse_ensemble_simple_x + rmse_ensemble_simple_y) / 2:.4f}\n")

# 2. Weighted averaging (weight by inverse RMSE)
weights_x = np.array([1 / m['val_rmse'] for m in metrics_x])
weights_x = weights_x / weights_x.sum()

weights_y = np.array([1 / m['val_rmse'] for m in metrics_y])
weights_y = weights_y / weights_y.sum()

ensemble_weighted_x = np.sum([w * predictions_x[m] for w, m in zip(weights_x, config.MODELS)], axis=0)
ensemble_weighted_y = np.sum([w * predictions_y[m] for w, m in zip(weights_y, config.MODELS)], axis=0)

rmse_ensemble_weighted_x = np.sqrt(mean_squared_error(y_val_x, ensemble_weighted_x))
rmse_ensemble_weighted_y = np.sqrt(mean_squared_error(y_val_y, ensemble_weighted_y))

print(f"📊 Weighted Averaging Ensemble:")
print(f"   X RMSE: {rmse_ensemble_weighted_x:.4f}")
print(f"   Y RMSE: {rmse_ensemble_weighted_y:.4f}")
print(f"   Average: {(rmse_ensemble_weighted_x + rmse_ensemble_weighted_y) / 2:.4f}\n")

print(f"   Weights X: {dict(zip(config.MODELS, [f'{w:.3f}' for w in weights_x]))}")
print(f"   Weights Y: {dict(zip(config.MODELS, [f'{w:.3f}' for w in weights_y]))}\n")

# 3. Stacking (use Ridge as meta-learner)
print("🏗️  Training stacking ensemble...")

# Prepare meta-features (predictions from base models)
meta_features_x = np.column_stack([predictions_x[m] for m in config.MODELS])
meta_features_y = np.column_stack([predictions_y[m] for m in config.MODELS])

# Train meta-learner
meta_learner_x = Ridge(alpha=1.0, random_state=42)
meta_learner_y = Ridge(alpha=1.0, random_state=42)

meta_learner_x.fit(meta_features_x, y_val_x)
meta_learner_y.fit(meta_features_y, y_val_y)

ensemble_stacking_x = meta_learner_x.predict(meta_features_x)
ensemble_stacking_y = meta_learner_y.predict(meta_features_y)

rmse_ensemble_stacking_x = np.sqrt(mean_squared_error(y_val_x, ensemble_stacking_x))
rmse_ensemble_stacking_y = np.sqrt(mean_squared_error(y_val_y, ensemble_stacking_y))

print(f"\n📊 Stacking Ensemble (Ridge meta-learner):")
print(f"   X RMSE: {rmse_ensemble_stacking_x:.4f}")
print(f"   Y RMSE: {rmse_ensemble_stacking_y:.4f}")
print(f"   Average: {(rmse_ensemble_stacking_x + rmse_ensemble_stacking_y) / 2:.4f}\n")

print("✅ Ensemble methods created")

In [ ]:
# Visualize ensemble comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Add ensemble results to comparison
ensemble_results = [
    ('Simple Avg', (rmse_ensemble_simple_x + rmse_ensemble_simple_y) / 2),
    ('Weighted Avg', (rmse_ensemble_weighted_x + rmse_ensemble_weighted_y) / 2),
    ('Stacking', (rmse_ensemble_stacking_x + rmse_ensemble_stacking_y) / 2)
]

# X coordinate comparison
individual_rmse_x = comparison['val_rmse_x'].tolist()
ensemble_rmse_x = [rmse_ensemble_simple_x, rmse_ensemble_weighted_x, rmse_ensemble_stacking_x]

all_models_x = [m.upper() for m in models_list] + ['Simple\nAvg', 'Weighted\nAvg', 'Stacking']
all_rmse_x = individual_rmse_x + ensemble_rmse_x

colors_x = ['steelblue'] * len(individual_rmse_x) + ['green', 'orange', 'red']
axes[0].bar(range(len(all_models_x)), all_rmse_x, color=colors_x, alpha=0.7)
axes[0].set_xticks(range(len(all_models_x)))
axes[0].set_xticklabels(all_models_x, rotation=45, ha='right')
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('X Coordinate: Individual vs Ensemble Models', fontsize=14, fontweight='bold')
axes[0].axhline(min(individual_rmse_x), color='red', linestyle='--', linewidth=1, label='Best individual')
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Y coordinate comparison
individual_rmse_y = comparison['val_rmse_y'].tolist()
ensemble_rmse_y = [rmse_ensemble_simple_y, rmse_ensemble_weighted_y, rmse_ensemble_stacking_y]

all_rmse_y = individual_rmse_y + ensemble_rmse_y

colors_y = ['steelblue'] * len(individual_rmse_y) + ['green', 'orange', 'red']
axes[1].bar(range(len(all_models_x)), all_rmse_y, color=colors_y, alpha=0.7)
axes[1].set_xticks(range(len(all_models_x)))
axes[1].set_xticklabels(all_models_x, rotation=45, ha='right')
axes[1].set_ylabel('RMSE', fontsize=12)
axes[1].set_title('Y Coordinate: Individual vs Ensemble Models', fontsize=14, fontweight='bold')
axes[1].axhline(min(individual_rmse_y), color='red', linestyle='--', linewidth=1, label='Best individual')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(config.OUTPUT_DIR / 'ensemble_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Ensemble comparison visualized")

## 9. Best Model Selection 🏆

Select the best model and save results.

In [ ]:
# Find best individual models
best_model_x = comparison.iloc[0]['model']
best_rmse_x = comparison.iloc[0]['val_rmse_x']

best_model_y_idx = comparison['val_rmse_y'].idxmin()
best_model_y = comparison.iloc[best_model_y_idx]['model']
best_rmse_y = comparison.iloc[best_model_y_idx]['val_rmse_y']

print("="*70)
print("BEST MODEL SELECTION")
print("="*70 + "\n")

print("🏆 Best Individual Models:")
print(f"   X coordinate: {best_model_x.upper()} (RMSE: {best_rmse_x:.4f})")
print(f"   Y coordinate: {best_model_y.upper()} (RMSE: {best_rmse_y:.4f})")
print(f"   Average: {(best_rmse_x + best_rmse_y) / 2:.4f}\n")

print("🎭 Best Ensemble:")
ensemble_avg_rmse = [
    ('Simple Averaging', (rmse_ensemble_simple_x + rmse_ensemble_simple_y) / 2),
    ('Weighted Averaging', (rmse_ensemble_weighted_x + rmse_ensemble_weighted_y) / 2),
    ('Stacking', (rmse_ensemble_stacking_x + rmse_ensemble_stacking_y) / 2)
]
best_ensemble = min(ensemble_avg_rmse, key=lambda x: x[1])
print(f"   {best_ensemble[0]} (Avg RMSE: {best_ensemble[1]:.4f})\n")

# Overall recommendation
all_options = [
    (f"Individual: {best_model_x}", (best_rmse_x + best_rmse_y) / 2),
    (f"Ensemble: {best_ensemble[0]}", best_ensemble[1])
]
overall_best = min(all_options, key=lambda x: x[1])

print("🎯 Recommendation:")
print(f"   Use: {overall_best[0]}")
print(f"   Expected RMSE: {overall_best[1]:.4f}\n")

# Save best models
print("💾 Saving best models...")
with open(config.OUTPUT_DIR / f'best_model_x_{best_model_x}.pkl', 'wb') as f:
    pickle.dump(models_x[best_model_x], f)

with open(config.OUTPUT_DIR / f'best_model_y_{best_model_y}.pkl', 'wb') as f:
    pickle.dump(models_y[best_model_y], f)

# Save feature names
with open(config.OUTPUT_DIR / 'feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

# Save summary
summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'best_model_x': best_model_x,
    'best_model_y': best_model_y,
    'best_rmse_x': float(best_rmse_x),
    'best_rmse_y': float(best_rmse_y),
    'best_ensemble': best_ensemble[0],
    'best_ensemble_rmse': float(best_ensemble[1]),
    'overall_recommendation': overall_best[0],
    'expected_rmse': float(overall_best[1]),
    'num_models_compared': len(config.MODELS),
    'num_features': len(feature_names)
}

import json
with open(config.OUTPUT_DIR / 'summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print(f"\n✅ Results saved to: {config.OUTPUT_DIR}")

---

## 🎉 Model Comparison Complete!

### Summary:

✅ **Models Compared**: Linear (Ridge, Lasso, ElasticNet), Tree-based (Random Forest, Gradient Boosting), Boosting (XGBoost, LightGBM)  
✅ **Cross-Validation**: K-Fold CV performed on top models  
✅ **Hyperparameter Tuning**: Grid search for Random Forest  
✅ **Ensemble Methods**: Simple averaging, weighted averaging, stacking  
✅ **Best Model Selected**: Based on validation RMSE  

### Key Findings:

1. **Best Individual Model**: Typically tree-based models (Random Forest, XGBoost, LightGBM)
2. **Ensemble Performance**: Often improves over individual models
3. **Trade-offs**: Linear models are fast but less accurate; tree models are slower but more accurate
4. **Feature Importance**: Consistent across tree-based models

### Next Steps:

1. Try LSTM models in `05_lstm_sequence_modeling.ipynb`
2. Generate final predictions in `06_prediction_and_evaluation.ipynb`
3. Experiment with feature selection to improve speed
4. Try more advanced ensembles (blending, stacking with different meta-learners)

---